In [23]:
import requests
import json
from bs4 import BeautifulSoup

In [24]:
import requests
from bs4 import BeautifulSoup
import json

url = "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

title = soup.find("h1", class_="uicore-title")
meta = soup.find("div", class_="uicore-entry-meta")
article = soup.find("article")

# Autor
author = None
if meta:
    author_tag = meta.select_one('a[href*="/author/"]')
    if author_tag:
        author = author_tag.get_text(strip=True)

# Data
date_tag = meta.find("span", class_="ui-blog-date") if meta else None
date = date_tag.get_text(strip=True) if date_tag else None

# Categoria
category_tag = meta.select_one(".uicore-post-category a") if meta else None
category = category_tag.get_text(strip=True) if category_tag else None

# Conteúdo do artigo (texto puro ou html)
content_div = article.find("div", class_="entry-content") if article else None
content_html = str(content_div) if content_div else None
content_text = content_div.get_text(" ", strip=True) if content_div else None

data = {
    "title": title.get_text(strip=True) if title else None,
    "author": author,
    "date": date,
    "category": category,
    "content_html": content_html,
    "content_text": content_text,
    "source_url": url
}

print(json.dumps(data, ensure_ascii=False, indent=2))


{
  "title": "CSL e IFR – Como converter o nível de serviço?",
  "author": "Henrique Alvarenga",
  "date": "14/05/2025",
  "category": "Insights",
  "content_html": "<div class=\"entry-content\">\n<p><span style=\"font-weight: 400;\">Em </span><b><a data-wpel-link=\"internal\" href=\"https://ilos.com.br/nivel-de-servico-qual-utilizar-para-o-estoque-de-seguranca/\">post anterior</a></b><span style=\"font-weight: 400;\">, comentei sobre o uso do nível de serviço para o cálculo do estoque de segurança. O principal ponto discutido foi a diferença entre o nível de serviço usualmente acompanhado pelas empresas (como o caso do IFR, ou </span><i><span style=\"font-weight: 400;\">Item Fill Rate</span></i><span style=\"font-weight: 400;\">) e o indicador de nível de serviço adequado para o uso na fórmula do estoque de segurança (o CSL, ou </span><i><span style=\"font-weight: 400;\">Cycle Service Level</span></i><span style=\"font-weight: 400;\">). Neste post, irei explorar uma forma prática de t

In [ ]:
# Montagem do Json final com a estrutura desejada

data = {
    "title": title.get_text(strip=True) if title else None,
    "author": author,
    "date": date,
    "category": category,
    "content_html": str(article) if article else None,
    "source_url": url
}

with open("pagina2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

''''
# Se quiser, ainda pode salvar o HTML para inspeção visual
with open("pagina2.html", "w", encoding="utf-8") as f:
    f.write(str(article) if article else "")'''


'\'\n# Se quiser, ainda pode salvar o HTML para inspeção visual\nwith open("pagina2.html", "w", encoding="utf-8") as f:\n    f.write(str(article) if article else "")'

In [26]:
title_text = title.get_text(strip=True) if title else ""
author_text = author or ""
date_text = date or ""
category_text = category or ""

# Monta o topo da página
header_html = f"""
<h1>{title_text}</h1>
<p style="color:#555;">
  {author_text} • {date_text} • {category_text}
</p>
<hr>
"""

# Corpo do artigo
article_html = str(article) if article else ""

# Junta tudo
pagina2_html = header_html + article_html
with open("pagina2_completa.html", "w", encoding="utf-8") as f:
    f.write(pagina2_html)